# Jupyter notebook utilities

The `evo-sdk-common` package contains helpful widgets that make it easier to interact with Evo services
in Jupyter notebooks. These widgets are built on top of the [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/) package,
which must be installed in order to use the widgets. The widgets are not required to use the `evo-sdk-common` package.

To install all dependencies for the notebook utilities, install `evo-sdk-common` with the `notebooks` optional dependencies selected:

```
pip install evo-sdk-common[notebooks]
```

## ServiceManagerWidget

The `ServiceManagerWidget` provides authorization and discovery in one convenient widget. Options that have been selected are stored in a `.env` file in a cache directory that can be configured with the `cache_location` parameter.

`ServiceManagerWidget.with_auth_code(...)` allows you to authenticate using the authorization code flow. This implementation will open a new browser tab to log in. The authorization code redirect will only work if the notebook is running on the user's local machine.

**Important:** With this authorization implementation, your user token is stored in plain text in the `.env` file. It is recommended to keep this file secure and not share it with others.

### Authorization Code flow

Using the `ServiceManagerWidget` with the authorization code flow is simple:

In [ ]:
from evo.notebooks import ServiceManagerWidget

manager = await ServiceManagerWidget.with_auth_code(
    client_id="your-client-id",
    redirect_url="http://localhost:3000/signin-callback",
    cache_location="./notebook-data/with-auth-code",
).login()

Once you have selected a valid organization, hub, and workspace, you can use the service manager widget to create `APIConnector` and `Environment` instances. These instances can be used to interact with Evo services. It is not necessary to open `APIConnector` instances, because the underlying transport has already been opened by the `ServiceManagerWidget`. The `Environment` cache location is set to the same cache location as the `ServiceManagerWidget` by default.

In [ ]:
from evo.common import BaseAPIClient

client = manager.create_client(BaseAPIClient)

You can access the notebook cache via the service manager widget.

In [ ]:
cache = manager.cache.get_location(manager.get_environment(), "<scope>")
print(f"{cache}: {cache.exists() = }")

manager.cache.clear_cache(manager.get_environment(), "<scope>")
print(f"{cache.exists() = }")

## FeedbackWidget

A `FeedbackWidget` is provided to display feedback a progress bar and messages in the notebook.

In [ ]:
import time

from evo.notebooks import FeedbackWidget

fb = FeedbackWidget("Loading...")

for i in range(100):
    n = round(i * 0.01, 2)
    if round(n, 1) == n:
        fb.progress(n, f"{n * 100:.0f}%")
    else:
        fb.progress(n)
    time.sleep(0.04)

fb.progress(1, "Done")